# Debugging code

This notebook serves as  testbed for the code written in the separate python files. Here I test the different functions with their inputs and use possible error messages to debug. 

In [3]:
# General imports
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp

# import code
from plotting import plotting
from solvers import solver

ImportError: cannot import name 'plotting' from 'plotting' (/Users/bertdepoorter/Nextcloud/Documents/School/Universiteit/CmpMethothods Astrophysics/Individual_CMfAA/plotting.py)

## Plotting class
Here I will test the different plotting functionality I have added to this class.

## solver class
This part is dedicated to testing the different solvers implemented. 